# Laboratorio 6 Parte 2

### Reducción de dimensión por extracción de características con PCA y LDA

### Universidad de Antioquia

### Facultad de Ingeniería

### Ingeniería de Sistemas

### Ude@ - 2018-II

#### Profesor: Antonio Tamayo Herrera

## Estudiantes

Nombre: Deiry Sofia Navas Muriel

Céduda: 1122140270

Nombre: David Alejandro Marín Alzate

Cédula: 1041325808

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

## Indicaciones

Este ejercicio tiene como objetivo implementar varias técnicas de extracción de características (PCA y LDA) y usar SVM para resolver un problema de clasificación multietiqueta o multiclase.

Para el problema de clasificación usaremos la siguiente base de datos: https://archive.ics.uci.edu/ml/datasets/Cardiotocography

#### Abstract: 
The dataset consists of measurements of fetal heart rate (FHR) and uterine contraction (UC) features on cardiotocograms classified by expert obstetricians.
	

#### Data Set Information:

2126 fetal cardiotocograms (CTGs) were automatically processed and the respective diagnostic features measured. The CTGs were also classified by three expert obstetricians and a consensus classification label assigned to each of them. Classification was both with respect to a morphologic pattern (A, B, C. ...) and to a fetal state (N, S, P). Therefore the dataset can be used either for 10-class or 3-class experiments.

#### Attribute Information:

LB - FHR baseline (beats per minute)

AC - # of accelerations per second

FM - # of fetal movements per second

UC - # of uterine contractions per second

DL - # of light decelerations per second

DS - # of severe decelerations per second

DP - # of prolongued decelerations per second

ASTV - percentage of time with abnormal short term variability

MSTV - mean value of short term variability

ALTV - percentage of time with abnormal long term variability

MLTV - mean value of long term variability

Width - width of FHR histogram

Min - minimum of FHR histogram

Max - Maximum of FHR histogram

Nmax - # of histogram peaks

Nzeros - # of histogram zeros

Mode - histogram mode

Mean - histogram mean

Median - histogram median

Variance - histogram variance

Tendency - histogram tendency

CLASS - FHR pattern class code (1 to 10)

NSP - fetal state class code (N=normal (1); S=suspect (2); P=pathologic (3))


Analice la base de datos, sus características, su variable de salida y el contexto del problema.

Antes de iniciar a ejecutar las celdas, debe tener instalada la librería mlxtend que usamos en la parte 1 del presente laboratorio. Si aún no la ha instalado, remítase a la guía del laboratorio anterior.

Analice y comprenda la siguiente celda de código donde se importan las librerías a usar y se carga la base de datos.

In [14]:
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold
from mlxtend.preprocessing import standardize
from mlxtend.feature_extraction import PrincipalComponentAnalysis as PCA
from mlxtend.feature_extraction import LinearDiscriminantAnalysis as LDA
import warnings
warnings.filterwarnings("always")
import time

#cargamos la bd de entrenamiento
db = np.loadtxt('DB_Fetal_Cardiotocograms.txt',delimiter='\t')  # Assuming tab-delimiter

X = db[:,0:22]

#Solo para dar formato a algunas variables
for i in range(1,7):
    X[:,i] = X[:,i]*1000

X = X
Y = db[:,22]

#Para darle formato de entero a la variable de salida

Y_l = []
for i in Y:
    Y_l.append(int(i))
Y = np.asarray(Y_l)

print ("Dimensiones de la base de datos de entrenamiento. dim de X: " + str(np.shape(X)) + "\tdim de Y: " + str(np.shape(Y)))
Y = Y.reshape(2126)

Dimensiones de la base de datos de entrenamiento. dim de X: (2126, 22)	dim de Y: (2126,)
[2 1 1 ... 2 2 1]


En la siguiente celda de código no tiene que completar nada. Analice, comprenda y ejecute el código y tenga en cuenta los resultados para completar la tabla que se le pide más abajo.

In [15]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

#Para calcular el costo computacional
tiempo_i = time.time()

#Creamos el clasificador SVM. Tenga en cuenta que el problema es multiclase. 
clf = svm.SVC(decision_function_shape='ovr', kernel='rbf', C = 100, gamma=0.0001)

#Implemetamos la metodología de validación

Errores = np.ones(10)
j = 0
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  

    #Aquí se entran y se valida el modelo sin hacer selección de características
    
    ######
    
    # Entrenamiento el modelo.
    model = clf.fit(X_train,y_train)

    # Validación del modelo
    ypred = model.predict(X_test)
    
    #######

    Errores[j] = classification_error(ypred, y_test)
    j+=1

print("\nError de validación sin aplicar extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")

#print str(ypred)
#print str(y_test) 


Error de validación sin aplicar extracción: 0.07712817787226504 +/- 0.05442325724156325


Tiempo total de ejecución: 0.6041691303253174 segundos.


## Ejercicio 1

1.1 Al aplicar PCA es necesario estandarizar los datos? Si, No y por qué? En qué consiste dicha estandarización?

R/: Si, por que los datos tienen que estar centrados y esto se hacer restando la media
    
1.2 La proyección de los datos que realiza PCA se hace buscando minimizar o maximizar algo? Explique.

R/: busca la proyeccion de los datos en un espacio de menor dimension donde la variabilidad de los datos es maxima.

## Ejercicio 2

En la siguiente celda, complete el código donde le sea indicado. Consulte la documentación oficial de la librería mlxtend para los métodos de extracción de características. https://rasbt.github.io/mlxtend/user_guide/feature_extraction/

In [37]:
#Feature Extraction Function
#Recibe 2 parámetros: 1. el tipo de método de extracción (pca o lda como string), 2. el número componentes (para pca)
#o el número de discriminantes (para lda)

#Para este laboratorio solo se le pedirá trabajar con PCA, LDA es opcional.

def extract_features(tipo, n):
    
    if tipo == 'pca':
    
        ext = PCA(n_components=n)
    
        return ext

    elif tipo == 'lda':
        
        ext = LDA(n_discriminants=n)
        
        return ext
    
    else:
        print("Ingrese un método válido (pca o lda)\n")

def pca_lda(typeModel='pca',n=2):
    #Para calcular el costo computacional
    tiempo_i = time.time()
    global X
    global Y
    print(len(Y))
    print(np.shape(X))
    #Estandarizamos los datos
    
    X = standardize(X)

    #Implemetamos la metodología de validación cross validation con 10 folds

    Errores = np.ones(10)
    j = 0
    kf = KFold(n_splits=10)

    for train_index, test_index in kf.split(X):
        
        #Aquí se aplica la extracción de características por PCA
        #Complete el código
        #Complete el código llamando el método extract_features. Tenga en cuenta lo que le pide el ejercicio 3.1
        ex = extract_features(tipo='pca', n=n)

        #Fit de PCA
        #Complete el código con el fit correspondiente
        ex = ex.fit(X)

        #Transforme las variables y genere el nuevo espacio de características de menor dimensión
        #complete el código aquí para hacer la transformación
        X_ex = ex.transform(X)

        #Aquí se aplica la extracción de características por LDA

        #OPCIONAL

        #Complete el código llamando el método extract_features. Tenga en cuenta lo que le pide el ejercicio 3.1
        #ex = extract_features(tipo='lda', n=n)
        
        #Fit de LDA
        #ex = ex.fit(X,Y)#Complete el código con el fit correspondiente

        #Transforme las variables y genere el nuevo espacio de características de menor dimensión
        #X_ex = ex.transform(X)#complete el código aquí para hacer la transformación


        #Se aplica CV-10

        X_train, X_test = X_ex[train_index], X_ex[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Aquí se entrena y se valida el modelo luego de aplicar extracción de características con PCA o LDA

        ######

        # Entrenamiento el modelo.
        model = clf.fit(X_train,y_train)

        # Validación del modelo
        ypred = model.predict(X_test)

        #######

        Errores[j] = classification_error(ypred, y_test)
        j+=1


    print("\nError de validación aplicando extracción: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

    print("\nEficiencia en validación aplicando extracción: " + str((1-np.mean(Errores))*100) + "%" )

    print ("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i) + " segundos.")
    
    error = round(np.mean(Errores),3)
    ic = round(np.std(Errores),3)
    t = round(time.time()-tiempo_i,2)
    return error,ic,t

    #print str(ypred)
    #print str(y_test)
pca_lda()

2126
(2126, 22)

Error de validación aplicando extracción: 0.22142793870139074 +/- 0.17001475908401714

Eficiencia en validación aplicando extracción: 77.85720612986093%


Tiempo total de ejecución: 0.9283039569854736 segundos.


(0.221, 0.17, 0.93)

In [38]:
features = [None,2,10,19,21]

errors = np.zeros(len(features))
ics = np.zeros(len(features))
timers = np.zeros(len(features))

errors[0] = 0.077
ics[0] = 0.054
timers[0] = 0.60

for i in range(1,len(features)):
    errors[i],ics[i],timers[i] = pca_lda(n=features[i])

2126
(2126, 22)

Error de validación aplicando extracción: 0.22142793870139074 +/- 0.17001475908401714

Eficiencia en validación aplicando extracción: 77.85720612986093%


Tiempo total de ejecución: 0.8342940807342529 segundos.
2126
(2126, 22)

Error de validación aplicando extracción: 0.09169324120825582 +/- 0.061564467997647386

Eficiencia en validación aplicando extracción: 90.83067587917442%


Tiempo total de ejecución: 0.4725189208984375 segundos.
2126
(2126, 22)

Error de validación aplicando extracción: 0.07193949862698203 +/- 0.04841386304281984

Eficiencia en validación aplicando extracción: 92.8060501373018%


Tiempo total de ejecución: 0.46214890480041504 segundos.
2126
(2126, 22)

Error de validación aplicando extracción: 0.07430463282841704 +/- 0.04391645244688941

Eficiencia en validación aplicando extracción: 92.5695367171583%


Tiempo total de ejecución: 0.4796321392059326 segundos.


## Ejercicio 3

3.1 En la celda de código anterior, varíe los parámetros correspondientes al número de componentes principales a tener en cuenta (use 2, 10, 19 y 21 componentes principales) para PCA y complete la siguiente tabla de resultados:

In [41]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Técnicas' : pd.Series(['SVM sin selección','SVM + PCA','SVM + PCA','SVM + PCA','SVM + PCA']),
    '# de componentes principales' : pd.Series(features)})
df_types["Error de validación"] = errors
df_types["IC (std)"] = ics
df_types["Tiempo de ejecución"] = timers
df_types.set_index(['Técnicas','# de componentes principales'], inplace=True)
#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)


In [42]:
qgrid_widget.get_changed_df()

Error de validación  IC (std)  \
Técnicas          # de componentes principales                                  
SVM sin selección NaN                                         0.077     0.054   
SVM + PCA         2.0                                         0.221     0.170   
                  10.0                                        0.092     0.062   
                  19.0                                        0.072     0.048   
                  21.0                                        0.074     0.044   

                                                Tiempo de ejecución  
Técnicas          # de componentes principales                       
SVM sin selección NaN                                          0.60  
SVM + PCA         2.0                                          0.83  
                  10.0                                         0.47  
                  19.0                                         0.46  
                  21.0                                         0.48

3.2 Analizando los resultados del punto anterior que puede decir de la viabilidad de aplicar PCA para hacer reducción de dimensión en este problema?

R/: si es viable por que con un espacio de 19 caracteristicas nos da un error en la validacion del 7.2% +- 4.8%, se le estarian quitando 3 dimensiones


3.3 Explique en sus palabras la principal ventaja que tiene LDA sobre PCA para resolver problemas de clasificación.

R/: el criterio de LDA realiza la proyeccion en la que las distancias entre las medias de cada clase es maxima con respecto a la otra.

3.3 Explique en sus palabras las diferencias que existen entre los métodos de selección de características y los métodos de extracción de características vistos en el curso.

R/:la principal deferencia es que selección de caracteristicas busca son las caracteristicas mas importantes y mas relvantes del problema de tal modo que su nivel predicción no disminuya, mientras que extracción de caracteristicas reduce es la dimension del problema haciendo proyecciones sobre un espacio menor en otras palabras transforma los datos.